In [35]:
import pandas as pd
import numpy as np

# Load the events dataset
data = pd.read_csv('C:/Users/nafla/Downloads/article/dataset/filtered_sessions.csv')
data.head()

,timestamp,visitorid,event,itemid,session_id,global_session_id,time_since_session_start,interaction_type_id
0,2015-09-11 20:49:49.439,0,view,285930,0,0:START:20150911204949:DUR:00327s,0.000,3
1,2015-09-11 20:52:39.591,0,view,357564,0,0:START:20150911204949:DUR:00327s,170.152,3
2,2015-09-11 20:55:17.175,0,view,67045,0,0:START:20150911204949:DUR:00327s,327.736,3
3,2015-08-30 06:39:38.318,6,view,253615,1,6:START:20150830063938:DUR:01015s,0.000,3
4,2015-08-30 06:40:23.805,6,view,344723,1,6:START:20150830063938:DUR:01015s,45.487,3


Convert raw timestamps into multiple features that capture different temporal aspects:
- Time of day: Extract hours, minutes, and even seconds.
- Day of week: Extract the day of the week from the timestamp.
- Month or Season: Extract the month or the season, which can be particularly useful for capturing seasonal behavior in user interactions.

In [36]:
# Convert 'timestamp' to datetime format if not already done
data['timestamp'] = pd.to_datetime(data['timestamp'])


data['hour'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['month'] = data['timestamp'].dt.month


In [37]:
# Encodeing time features using sin and cosine 
data['hour_sin'] = np.sin(2 * np.pi * data['hour']/24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour']/24)
data['day_sin'] = np.sin(2 * np.pi * data['day_of_week']/7)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_week']/7)

# Optionally, view some of the data to confirm it looks correct
print(data[['hour', 'hour_sin', 'hour_cos', 'day_of_week', 'day_sin', 'day_cos']].head())


   hour  hour_sin      hour_cos  day_of_week   day_sin   day_cos
0    20 -0.866025  5.000000e-01            4 -0.433884 -0.900969
1    20 -0.866025  5.000000e-01            4 -0.433884 -0.900969
2    20 -0.866025  5.000000e-01            4 -0.433884 -0.900969
3     6  1.000000  6.123234e-17            6 -0.781831  0.623490
4     6  1.000000  6.123234e-17            6 -0.781831  0.623490


In [38]:
# finding unique users
unique_users = data['visitorid'].unique()

# Set a random seed for reproducibility
np.random.seed(42)

# Calculate the number of users to sample
sample_size = int(len(unique_users) * 0.40)

# Randomly sample user IDs
sampled_users = np.random.choice(unique_users, size=sample_size, replace=False)

In [39]:
# Filter the original data to keep only the sessions of sampled users
sampled_data = data[data['visitorid'].isin(sampled_users)]

# Optional: Verify the results
sampled_data.head()
print(f"Number of sessions in the sampled data: {sampled_data['global_session_id'].nunique()}")
print(f"Number of users in the sampled data: {sampled_data['visitorid'].nunique()}")


Number of sessions in the sampled data: 127382
Number of users in the sampled data: 111284


In [40]:
sampled_data.head()

,timestamp,visitorid,event,itemid,session_id,global_session_id,time_since_session_start,interaction_type_id,hour,day_of_week,month,hour_sin,hour_cos,day_sin,day_cos
3,2015-08-30 06:39:38.318,6,view,253615,1,6:START:20150830063938:DUR:01015s,0.000,3,6,6,8,1.000000,6.123234e-17,-0.781831,0.623490
4,2015-08-30 06:40:23.805,6,view,344723,1,6:START:20150830063938:DUR:01015s,45.487,3,6,6,8,1.000000,6.123234e-17,-0.781831,0.623490
5,2015-08-30 06:54:09.385,6,view,344723,1,6:START:20150830063938:DUR:01015s,871.067,3,6,6,8,1.000000,6.123234e-17,-0.781831,0.623490
6,2015-08-30 06:56:33.838,6,view,344723,1,6:START:20150830063938:DUR:01015s,1015.520,3,6,6,8,1.000000,6.123234e-17,-0.781831,0.623490
11,2015-07-02 23:16:15.173,22,view,86411,0,22:START:20150702231615:DUR:00023s,0.000,3,23,3,7,-0.258819,9.659258e-01,0.433884,-0.900969


In [41]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Assuming 'sampled_data' is your DataFrame from the previous step
sampled_data['time_since_session_start'] = scaler.fit_transform(sampled_data[['time_since_session_start']])


C:\Users\nafla\AppData\Local\Temp\ipykernel_11284\2524327454.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['time_since_session_start'] = scaler.fit_transform(sampled_data[['time_since_session_start']])


In [42]:
from sklearn.preprocessing import LabelEncoder

# Initialize the Label Encoder
encoder = LabelEncoder()

# Encode the itemid
sampled_data['itemid'] = encoder.fit_transform(sampled_data['itemid'])
# Encode month
sampled_data['month'] = encoder.fit_transform(sampled_data['month'])


C:\Users\nafla\AppData\Local\Temp\ipykernel_11284\1288704688.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['itemid'] = encoder.fit_transform(sampled_data['itemid'])
C:\Users\nafla\AppData\Local\Temp\ipykernel_11284\1288704688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['month'] = encoder.fit_transform(sampled_data['month'])


In [43]:
sampled_data.head()

,timestamp,visitorid,event,itemid,session_id,global_session_id,time_since_session_start,interaction_type_id,hour,day_of_week,month,hour_sin,hour_cos,day_sin,day_cos
3,2015-08-30 06:39:38.318,6,view,39276,1,6:START:20150830063938:DUR:01015s,0.000000,3,6,6,3,1.000000,6.123234e-17,-0.781831,0.623490
4,2015-08-30 06:40:23.805,6,view,53453,1,6:START:20150830063938:DUR:01015s,0.009186,3,6,6,3,1.000000,6.123234e-17,-0.781831,0.623490
5,2015-08-30 06:54:09.385,6,view,53453,1,6:START:20150830063938:DUR:01015s,0.175914,3,6,6,3,1.000000,6.123234e-17,-0.781831,0.623490
6,2015-08-30 06:56:33.838,6,view,53453,1,6:START:20150830063938:DUR:01015s,0.205087,3,6,6,3,1.000000,6.123234e-17,-0.781831,0.623490
11,2015-07-02 23:16:15.173,22,view,13332,0,22:START:20150702231615:DUR:00023s,0.000000,3,23,3,2,-0.258819,9.659258e-01,0.433884,-0.900969


In [44]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# data['itemid'] = scaler.fit_transform(data[['itemid']])

# Convert 'interaction_type_id' to categorical (one-hot encoded format)
data['interaction_type_id'] = to_categorical(data['interaction_type_id'])

# Assuming 'sampled_data' is already loaded and contains a 'timestamp' column
sampled_data.sort_values('timestamp', inplace=True)

# Calculate indices for splitting
total_count = len(sampled_data)
train_end = int(total_count * 0.80)  # Adjusted to use 80% of the data for training

# Split the data
train_data = sampled_data[:train_end]
test_data = sampled_data[train_end:]  # Rest of the data for testing

C:\Users\nafla\AppData\Local\Temp\ipykernel_11284\155815241.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data.sort_values('timestamp', inplace=True)


In [45]:
# Verify the minimum sequence length in validation and test sets
min_test_length = test_data.groupby('global_session_id').size().min()

print(f"Minimum interactions in test sequences: {min_test_length}")


Minimum interactions in test sequences: 1


In [46]:
train_data.head()

,timestamp,visitorid,event,itemid,session_id,global_session_id,time_since_session_start,interaction_type_id,hour,day_of_week,month,hour_sin,hour_cos,day_sin,day_cos
384073,2015-05-03 03:00:04.384,693516,addtocart,46087,0,693516:START:20150503030004:DUR:00025s,0.000000,1,3,6,0,0.707107,0.707107,-0.781831,0.62349
384074,2015-05-03 03:00:26.228,693516,view,46087,0,693516:START:20150503030004:DUR:00025s,0.004411,3,3,6,0,0.707107,0.707107,-0.781831,0.62349
384075,2015-05-03 03:00:29.427,693516,addtocart,46087,0,693516:START:20150503030004:DUR:00025s,0.005057,1,3,6,0,0.707107,0.707107,-0.781831,0.62349
144845,2015-05-03 03:01:07.992,260113,view,19463,0,260113:START:20150503030107:DUR:00362s,0.000000,3,3,6,0,0.707107,0.707107,-0.781831,0.62349
50695,2015-05-03 03:01:14.535,90447,view,37466,0,90447:START:20150503030114:DUR:00116s,0.000000,3,3,6,0,0.707107,0.707107,-0.781831,0.62349


 ### Group Data by Sessions:
 The data is grouped by global_session_id, which represents individual sessions. The agg(list) aggregates all the columns into lists, effectively creating a list of interactions for each session.

 ### Create Sequences for Training:
 Inputs: The interaction data is combined into a matrix (inputs) where each row corresponds to an interaction featuring the interaction type ID, item ID, and normalized time since session start.
 X (features): The sequences of interactions up to the second last are stored in X.
 y (labels/targets): The sequences from the second interaction onward are stored in y. This setup makes the RNN learn to predict the next interaction based on the previous ones

In [47]:

# Group data by 'global_session_id' and collect sequences
grouped = train_data.groupby('global_session_id').agg(list)

# Create sequences for training
X = []
y = []
for session_id, group in grouped.iterrows():
    inputs = np.column_stack((group['interaction_type_id'], group['itemid'], group['time_since_session_start'], group['hour_sin'], group['hour_cos'], group['day_sin'], group['day_cos']))
    X.append(inputs[:-1])
    y.append(inputs[1:])


### Pad Sequences:
Padding: Since sessions can have varying lengths, padding is applied to standardize their lengths, enabling batch processing by the neural network.

In [48]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad sequences
X_padded = pad_sequences(X, padding='post', dtype='float32')
y_padded = pad_sequences(y, padding='post', dtype='float32')

### Define and Compile the RNN Model:

LSTM Layers: Two LSTM layers are used, both returning sequences to ensure that the model outputs a sequence that matches the input sequence in length.

TimeDistributed Layer: This applies a dense layer to each time step independently, predicting the interaction type, item ID, and time for each step.

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import GRU


# Define model architecture
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(None, X_padded.shape[2])),
    GRU(50, return_sequences=True),
    TimeDistributed(Dense(X_padded.shape[2], activation='softmax'))  # Predict the next interaction
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, None, 50)       │         8,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, None, 50)       │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, None, 7)        │           357 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,507 (95.73 KB)

 Trainable params: 24,507 (95.73 KB)

 Non-trainable params: 0 (0.00 B)

### Training the Model:
The model is trained for 10 epochs, using 20% of the data for validation to monitor overfitting.

In [57]:
# Train the model
history = model.fit(X_padded, y_padded, epochs=10, validation_split=0.2)

Epoch 1/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.4757 - loss: 1305.5811 - val_accuracy: 0.4856 - val_loss: 16.6951
Epoch 2/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4830 - loss: 16.5188 - val_accuracy: 0.4856 - val_loss: 16.1699
Epoch 3/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.4825 - loss: 16.0637 - val_accuracy: 0.4856 - val_loss: 15.9533
Epoch 4/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.4825 - loss: 15.9445 - val_accuracy: 0.4856 - val_loss: 15.9525
Epoch 5/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.4834 - loss: 16.0410 - val_accuracy: 0.4856 - val_loss: 15.9521
Epoch 6/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4821 - loss: 15.9667 - val_accuracy: 0.4856 - val_loss: 15.9627
Epoch 7/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4823 - loss: 16.0050 - val_accuracy: 0.4856 - val_loss: 15.9509
Epoch 8/10
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.48

In [51]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_padded, y_padded)
print(f"Test Accuracy: {test_accuracy:.2f}")

3181/3181 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4837 - loss: 15.9917
Test Accuracy: 0.48
